<a href="https://colab.research.google.com/github/erena29/Data-Analysis-SQL/blob/main/Real%20Estate%20Property%20Management%20Analysis/Real_Estate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Property Management Data Analysis with SQL**

## Data Import and Database Setup

In [1]:
import pandas as pd
import sqlite3

In [2]:
!pip install prettytable==3.6.0

  Attempting uninstall: prettytable
    Found existing installation: prettytable 3.12.0
    Uninstalling prettytable-3.12.0:
      Successfully uninstalled prettytable-3.12.0


In [3]:
# Load the SQL extension
%load_ext sql

# Create a SQLite database
%sql sqlite://

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
expense = pd.read_csv('/content/drive/MyDrive/Dataset/real_estate/Expense.csv')
property = pd.read_csv('/content/drive/MyDrive/Dataset/real_estate/Property.csv')
sales = pd.read_csv('/content/drive/MyDrive/Dataset/real_estate/Sales.csv')

In [6]:
%sql drop table if exists exp, property, sales;
# Persist the DataFrames as tables in SQLi
%sql --persist expense
%sql --persist property
%sql --persist sales

 * sqlite://
(sqlite3.OperationalError) near ",": syntax error
[SQL: drop table if exists exp, property, sales;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
 * sqlite://
 * sqlite://
 * sqlite://


'Persisted sales'

## Sample Data

In [ ]:
%%sql
SELECT * FROM sales LIMIT 5

 * sqlite://
Done.


index,SaleID,PropertyID_Sale,SaleDate,MeansofSales,ClientID,PaymentStatus
0,1,1,2023-07-14,Broker,131,Paid
1,2,3,2024-04-28,Direct,198,Paid
2,3,4,2022-01-05,Online,123,Pending
3,4,5,2022-02-04,Online,198,Paid
4,5,6,2022-06-17,Online,198,Pending


In [ ]:
%%sql
SELECT * FROM property LIMIT 5;

 * sqlite://
Done.


index,PropertyID,Type,SquareFootage,Price,Status
0,10,Townhouse,1200,200000,Sold
1,128,Townhouse,1800,200000,Sold
2,129,Condo,2500,800000,Sold
3,185,Apartment,1000,1000000,Sold
4,3,Townhouse,750,700000,Sold


In [ ]:
%%sql
SELECT * FROM expense LIMIT 5;

 * sqlite://
Done.


index,ExpenseID,PropertyID_Expense,ExpenseType,Amount
0,1,1,Maintenance,100000
1,2,3,Maintenance,300000
2,3,4,Maintenance,250000
3,4,5,Renovation,50000
4,5,6,Maintenance,250000


## SQL Analysis

### **1. Yearly Financial Overview and Property Sales**



In [7]:
%%sql
SELECT
  strftime('%Y', SaleDate) AS Year, --mysql: EXTRACT(YEAR FROM s.SaleDate)
  SUM(Amount) AS Total_Expense,
  SUM(Price) - SUM(Amount) AS Total_Income,
  COUNT(*) AS Properties_Sold
FROM sales AS s
LEFT JOIN property AS p ON s.PropertyID_Sale = p.PropertyID
LEFT JOIN expense AS e ON s.PropertyID_Sale = e.PropertyID_Expense
GROUP BY Year
ORDER BY Year;

 * sqlite://
Done.


Year,Total_Expense,Total_Income,Properties_Sold
2022,10350000,19550000,56
2023,10550000,30250000,60
2024,8700000,16800000,44


The financial overview shows a **significant increase in total income and properties sold from 2022 to 2023**, while total expenses remained relatively stable. However, in 2024, the data is incomplete, and the decrease in income and properties sold may indicate lower values due to missing data from several months.

### **2. Months with Repeated Top 3 Revenue for Each Year**
Identifies the months that appear more than once in the top 3 revenue  months each year. It helps uncover patterns of consistent high revenue months, providing insights into recurring trends that can guide future sales strategies and planning.

In [8]:
%%sql
-- Aggregate Monthly Revenue by Year and Month
WITH MonthlyRevenue AS (
  SELECT
    strftime('%Y', SaleDate) AS Year,
    strftime('%m', SaleDate) AS Month,
    SUM(Price) AS Revenue
  FROM sales AS s
  LEFT JOIN property AS p
    ON s.PropertyID_Sale = p.PropertyID
  GROUP BY Year, Month
)

SELECT
  Month
FROM (
  -- Rank months within each year by revenue
  SELECT
    Year,
    Month,
    Revenue,
    ROW_NUMBER() OVER (PARTITION BY Year ORDER BY Revenue DESC) AS Rank
  FROM MonthlyRevenue
)
WHERE Rank <= 3
GROUP BY Month
HAVING COUNT(Month) > 1
ORDER BY Month;

 * sqlite://
Done.


Month
04
07


`July` and `April` are the months with repeated appearances in the top 3 revenue rankings for each year, indicating that these months consistently perform well in terms of sales. To maximize this trend, consider **focusing marketing efforts and special promotions** in these months to sustain or further enhance revenue.

### **3. Months with Repeated Revenue Below the Yearly Average**
Identifies the months that appear more than once with revenue consistently below the yearly average.

In [9]:
%%sql
-- Calculate the Average Yearly Revenue for each year
WITH avg_year AS (
  SELECT
    Year,
    ROUND(AVG(Monthly_Revenue), 2) AS Avg_Yearly_Revenue
  FROM (
    SELECT
      strftime('%Y', SaleDate) AS Year,
      strftime('%m', SaleDate) AS Month,
      SUM(Price) AS Monthly_Revenue
    FROM sales AS s
    LEFT JOIN property AS p
      ON s.PropertyID_Sale = p.PropertyID
    GROUP BY Year, Month
  ) AS Monthly_Revenues
  GROUP BY Year
  ORDER BY Year
)
--Filter the months where revenue is below the yearly average
, filtered_months AS(
  SELECT
    strftime('%Y', SaleDate) AS Year,
    strftime('%m', SaleDate) AS Month,
    SUM(Price) AS Revenue
  FROM sales AS s
  LEFT JOIN property AS p
    ON s.PropertyID_Sale = p.PropertyID
  JOIN avg_year AS avg
    ON strftime('%Y', SaleDate) = avg.Year
  GROUP BY Year, Month, avg.Avg_Yearly_Revenue
  HAVING SUM(p.Price) < avg.Avg_Yearly_Revenue
  ORDER BY Month DESC
)
--Select months that appear more than once
SELECT Month
FROM filtered_months
GROUP BY Month
HAVING COUNT(Month) > 1
ORDER BY Month;

 * sqlite://
Done.


Month
01
05
06
08
10
11


The analysis shows that months such as `January`, `May`, `June`, `August`, `October`, and `November` consistently generate revenue below the yearly average, indicating periods of weaker performance. It is recommended to **investigate external factors impacting sales** during these months to develop targeted strategies for improvement.

### **4. Revenue by Property Type and Year**

In [11]:
%%sql
SELECT
  Type AS Property_Type,
  SUM(CASE WHEN strftime('%Y', SaleDate) = '2022' THEN p.Price ELSE 0 END) AS Revenue_2022,
  SUM(CASE WHEN strftime('%Y', SaleDate) = '2023' THEN p.Price ELSE 0 END) AS Revenue_2023,
  SUM(CASE WHEN strftime('%Y', SaleDate) = '2024' THEN p.Price ELSE 0 END) AS Revenue_2024,
  SUM(Price) AS Total_Revenue
FROM sales AS s
LEFT JOIN property AS p
  ON s.PropertyID_Sale = p.PropertyID
GROUP BY Property_Type
ORDER BY Total_Revenue DESC;

 * sqlite://
Done.


Property_Type,Revenue_2022,Revenue_2023,Revenue_2024,Total_Revenue
Condo,6500000,16800000,7500000,30800000
Single Family,9900000,7800000,7300000,25000000
Apartment,7400000,8900000,6900000,23200000
Townhouse,6100000,7300000,3800000,17200000


`Condos` generated **the highest total revenue** over the years, with a significant increase in 2023, while `Townhouses` experienced **the lowest total revenue**, particularly in 2024. To optimize revenue, focus on increasing sales efforts for townhouses and consider strategic marketing for condos, given their strong performance.

### **5. Percentage of Sold Properties and Net Income by Price Category**

In [12]:
%%sql
SELECT
    CASE
        WHEN Price BETWEEN 0 AND 300000 THEN '1. Low'
        WHEN Price BETWEEN 300001 AND 500000 THEN '2. Affordable'
        WHEN Price BETWEEN 500001 AND 800000 THEN '3. Mid-Range'
        WHEN Price > 800000 THEN '4. High'
    END AS PriceCategory,
    ROUND(AVG(Price),2) AS Average_Price,
    ROUND(COUNT(CASE WHEN Status = 'Sold' THEN 1 END) * 100.0 / COUNT(*), 2) AS Sold_Percentage,
    SUM(Price) - SUM(Amount) AS Total_Income
FROM property AS p
LEFT JOIN expense AS e ON p.PropertyID = e.PropertyID_Expense
GROUP BY PriceCategory
ORDER BY PriceCategory

 * sqlite://
Done.


PriceCategory,Average_Price,Sold_Percentage,Total_Income
1. Low,250000.0,78.0,4600000
2. Affordable,450000.0,69.57,15400000
3. Mid-Range,677777.78,90.74,27400000
4. High,950000.0,80.0,40300000


The `Mid-Range` category has **the highest demand** with the largest sold percentage, while the `High` category **leads in total income**. This suggests a balanced portfolio should **focus on increasing** `Mid-Range` and `High` properties to maximize both sold percentage and revenue. The `Affordable` category, while lower in sold percentage, **still provides significant income** and may attract budget-conscious buyers.

### **6. Frequency of Sales by Means of Sale**

In [ ]:
%%sql
SELECT
  MeansofSales,
  COUNT(MeansofSales) AS Count
FROM sales
GROUP BY MeansofSales

 * sqlite://
Done.


MeansofSales,Count
Broker,61
Direct,45
Online,54


`Broker` sales had the **highest frequency**, followed by `Online` sales, with direct sales being the least frequent. To boost overall sales, **consider enhancing** `Online` and `Direct` sales channels, especially since they have room for growth compared to `Broker` sales.

### **7. Comparison of Payment Status by Means of Sale**

In [ ]:
%%sql
SELECT
    MeansofSales,
    COUNT(CASE WHEN PaymentStatus = 'Paid' THEN 1 END) AS Paid_Count,
    COUNT(CASE WHEN PaymentStatus = 'Pending' THEN 1 END) AS Pending_Count,
    ROUND(
        (COUNT(CASE WHEN PaymentStatus = 'Paid' THEN 1 END) * 100.0) / COUNT(*),
        2
    ) AS Paid_Percentage
FROM sales
GROUP BY MeansofSales
ORDER BY MeansofSales;

 * sqlite://
Done.


MeansofSales,Paid_Count,Pending_Count,Paid_Percentage
Broker,23,38,37.7
Direct,17,28,37.78
Online,24,30,44.44


`Online` sales show **the highest percentage of paid transactions** at 44.44%, while `Broker` and `Direct` sales have similar paid percentages around 37.7% and 37.78%, respectively. To improve payment rates, **focusing on enhancing the payment collection process** for `Broker` and `Direct` sales could help achieve more consistent paid transactions. This can be achieved by implementing automated payment reminders, offering flexible payment options (such as installments or early-bird discounts), and establishing clearer communication regarding payment deadlines and consequences for late payments.

### **8. Total Expenses by Property Type and Expense Category**

In [14]:
%%sql
SELECT
    Type AS Property_Type,
    SUM(CASE WHEN ExpenseType = 'Maintenance' THEN Amount END) AS Maintenance,
    SUM(CASE WHEN ExpenseType = 'Property Taxes' THEN Amount END) AS PropTaxes,
    SUM(CASE WHEN ExpenseType = 'Renovation' THEN Amount END) AS Renovation
FROM expense e
LEFT JOIN property p ON e.PropertyID_Expense = p.PropertyID
GROUP BY Property_Type

 * sqlite://
Done.


Property_Type,Maintenance,PropTaxes,Renovation
Apartment,2400000,3850000,1700000
Condo,3100000,2300000,1900000
Single Family,3350000,3450000,1600000
Townhouse,2600000,1900000,1450000


`Single Family` properties have **the highest total expenses** in the `Maintenance` and `Property Tax` categories, while `Condos` incur **the highest renovation costs**. Overall, `Maintenance` is **the largest expense** category across all property types, followed by `Property taxes` and then `Renovations`. To optimize expenses, it is recommended to focus on reducing maintenance costs for `Single Family` properties and `Condos`, as well as controlling `Property Tax` expenses across all property types. This approach could lead to significant cost efficiencies across expense categories.